# Геном вируса

Я выбрал вирус `Ralstonia phage Cimandef`. Это `dsDNA` вирус бактериофаг. Его RefSeq геном [NC_054956](https://www.ncbi.nlm.nih.gov/nuccore/NC_054956)

In [6]:
# скачиваем геном
./ncbidl.sh NC_054956.1.fasta
# скачиваем разметку генами
./ncbidl.sh NC_054956.1.gff3

: 1

# Участки Z-ДНК

In [7]:
# запускаем zhunt
./zhunt2 12 8 12 NC_054956.1.fasta

dinucleotides 12
min/max 8 12
min/max 8 12
operating on NC_054956.1.fasta
calculating zscore
opening NC_054956.1.fasta
inputting sequence
opening NC_054956.1.fasta.Z-SCORE

 run time=9 sec
use min/max 8 12
analyzing_zscore
opening NC_054956.1.fasta.Z-SCORE
opening NC_054956.1.fasta
inputting sequence

: 1

Получилась таблица с полями `Start, End, 3, 4, 5, Z-Score, Sequence, 8`.
Поля 3, 4, 5 и 8 нам не нужны.
Первой строкой идёт заголовок с описанием всего генома, и он нам тоже не нужен.

In [8]:
head NC_054956.1.fasta.Z-SCORE

NC_054956.1.fasta 55186 8 12
1 17 16  31.409  26.681 2.593198e-01 ncatnaagcanctgna   SASAASASASASASAS
2 18 16  32.446   9.846 1.522824e-01 catnaagcanctgnat   ASSASASASASASASA
3 19 16  31.425  26.041 2.575059e-01 atnaagcanctgnatt   SAASASASASASASAS
4 28 24  34.614  19.197 3.409497e-02 tnaagcanctgnattgttgggcgg   SASASASASASASASASASASASA
5 21 16  31.326   3.116 2.693018e-01 naagcanctgnattgt   ASASASASASSASASA
6 30 24  30.270   1.854 4.104079e-01 aagcanctgnattgttgggcgggc   SASASASASASASASASASASASA
7 31 24  27.814  11.324 4.010115e+00 agcanctgnattgttgggcgggcc   ASASASASASASASASASASASAS
8 30 22  27.495   7.488 4.679213e+00 gcanctgnattgttgggcgggc   SASASASASASASASASASASA
9 31 22  27.121  18.057 5.685565e+00 canctgnattgttgggcgggcc   ASASASSASAASASASASASAS

: 1

Теперь нужно отфильтровать таблицу чтобы остались только участки, у которых Z-Score больше некоторого порога.
Я выбрал порог $400$.

In [9]:
# фильтруем Z-Score (6ая колонка) > 400 и считаем количество совпадений
awk '$6 > 400' NC_054956.1.fasta.Z-SCORE | wc

   2181   17448  185028

: 1

Получилось 2181 совпадений. Это много. А нам нужно искать в какие гены оно всё попадает. 
Нахождение пересечения разметок можно сделать через `bedtools`.
Но они работают только с форматом `.bed`.
К счастью это очень простой формат.
В нём важны только первые три колонки.
`Хромосома/геном, Начало, Конец`

In [10]:
# скачиваем bedtools
curl -sL 'https://github.com/arq5x/bedtools2/releases/download/v2.30.0/bedtools.static.binary' -o bedtools
chmod +x bedtools

: 1

Сначало надо сконвертировать `.gff3` в `.bed`. Для этого существуют автоматические инструменты но их сложно скачивать и они создают таблице с кучей лишней информации, котороя нам не нужна. Лучше просто выбрать нужные колонки из `.gff3` через `awk`.

In [11]:
# нам нужны 1ая (Геном), 4ая (Начало), 5ая (Конец) и 11ая (Название гена) колонки
# также нам нужны только те ряды, где 3ая колонка (тип последовательности) — ген
awk -v OFS='\t' -F'[;\t ]' '$3 == "gene" {print $1, $4, $5, $11}' NC_054956.1.gff3 > genes.bed
wc genes.bed

  66  264 2578 genes.bed

: 1

Всего получилось 66 генов.
Теперь тоже самое надо сделать с `.Z-SCORE`-ом. Тут у нас нет выбора и придётся делать ручками.

In [12]:
# тут берём заглушку с "NC_054956.1", 1ую и 2ую колонку
# а также фильтруем  Z-Score > 400
awk -v OFS='\t' '$6 > 400 {print "NC_054956.1", $1, $2}' NC_054956.1.fasta.Z-SCORE > zdna.bed

: 1

И только теперь можно искать пересечения.

In [13]:
# ищём уникальные пересечения и сортируем их по названию гена (4ой колонке) 
./bedtools intersect -a genes.bed -b zdna.bed -wa | uniq | sort -k4 > zgenes.bed
wc zgenes.bed

  56  224 2190 zgenes.bed

: 1

Всего zDNA нашлась в 56 из 66 генов.

In [14]:
head zgenes.bed

NC_054956.1	38902	39627	Name=cysH
NC_054956.1	12075	12881	Name=dnaA
NC_054956.1	3185	4249	Name=dnaN
NC_054956.1	428	739	Name=KMC44_gp02
NC_054956.1	903	1511	Name=KMC44_gp03
NC_054956.1	1508	1738	Name=KMC44_gp04
NC_054956.1	2171	2590	Name=KMC44_gp06
NC_054956.1	2587	3123	Name=KMC44_gp07
NC_054956.1	4274	4501	Name=KMC44_gp09
NC_054956.1	4507	5457	Name=KMC44_gp10

: 1

zDNA попала на гены cysH, dnaA и dnaN, а также на 53 неопознанных генов.

# Квадруплексы 

Квадруплексы находятся с помощью regex-а применённого к последовательности нуклеотидов. Для этого надо достать эту последовательность из `.fasta` файла.

In [15]:
# конвертируем fasta файл в непрерывную последовательность нуклеотидов, чтобы можно было запускать на нём regex-ы
tail NC_054956.1.fasta -n+2 | tr -d '\n' > dna.txt

: 1

In [16]:
grep -Pbo '(G{3,}[ATGC]{1,9}){3,}G{3,}' dna.txt

32946:GGGCGCTACGCTGGGCAGCGGGCACTTCGGCGGG

: 1

Нашлось только одно совпадение. `.bed` файл можно составить ручками.
Оно начинается с 32946 и 34 нуклиатида в длинну.

In [17]:
echo -e "NC_054956.1\t32946\t$((32946 + 34))" | tee quad.bed

NC_054956.1	32946	32980

: 1

In [18]:
./bedtools intersect -a genes.bed -b quad.bed -wa

NC_054956.1	32000	33706	Name=KMC44_gp23

: 1

Квадроплекс попадает на неопознанный ген KMC44_gp23.